In [1]:
import pandas as pd
import os
from surprise import Dataset, KNNBasic, Reader, accuracy, SVD
from surprise.model_selection import cross_validate, PredefinedKFold

## Descrição do Problema. 
Este projeto tem como principal objetivo desenvolver um sistema de apoio a recomendação. Para está analise iremos utilizar a base de dados [MovieLens](https://grouplens.org/datasets/movielens/) que consiste numa base de dados onde os dados foram coletados através do site da MovieLens (movielens.umn.edu) durante o período de sete meses a partir de 19 de setembro, 1997 até 22 de abril de 1998. Esses dados foram limpos - usuários que tinha menos de 20 classificações ou não tinha demografia completa informações foram removidas deste conjunto de dados. Descrições detalhadas dê o arquivo de dados pode ser encontrado no final deste arquivo.
Á partir desses dados foi desenvolvido um sistema de recomendação baseado em dois algoritimos que explicamos abaixo.

## Leitura do Conjunto de Dados
Abaixo o algoritimo é reponsavel pela leitura de um conjunto especifico da base de dados [ml-100k](https://grouplens.org/datasets/movielens/), onde iremos utilizar o conjunto especifico de teste e a base de intems, considerando os 1000 usuario presentes. 

In [2]:
items_stream = open('../ml-100k/u.item', 'r')
item_data = items_stream.read().split('\n')
item_data = list(map(lambda item: item.split('|')[:2], item_data))
items_stream.close()

In [3]:
database = pd.read_csv('../ml-100k/u1.base.csv')
user_set = set(database.user_id)
item_set = set(database.item_id)
not_watch = {user: item_set.difference(database.query('user_id == %s' %(user)).item_id) for user in user_set}

In [4]:
files_dir = os.path.expanduser('../ml-100k/')
reader = Reader('ml-100k')

## Leitura do Conjunto de Teste
Consideramos para a o conjunto de teste a base 1 contida no conjunto dos dados.

In [5]:
train_file = files_dir + 'u%d.base'
test_file = files_dir + 'u%d.test'
folds_files = [(train_file % i, test_file % i) for i in [1]]

data = Dataset.load_from_folds(folds_files, reader=reader)
pkf = PredefinedKFold()

## Algoritmos utilizados 
Para está analise utilizamos dois algoritimos diferente para observar o resultados, ambos os algoritimos estão presentes na biblioteca surprise. São eles:
+ KNN
+ SVD

### KNN
+ A ideia principal do KNN é determinar o rótulo de classificação de uma amostra baseado nas amostras vizinhas advindas de um conjunto de treinamento.
+ Passos:
    + 1-Escolha um vértice arbitrário como vértice atual.
    + 2-Descubra a aresta de menor peso que seja conectada ao vértice atual e a um vértice não visitado V.
    + 3-Faça o vértice atual ser V.
    + 4-Marque V como visitado.
    + 5-Se todos os vértices no domínio estiverem visitados, encerre o algoritmo.
    + 6-Se não vá para o passo 2.
+ Mais detalhe sobre a formulação e como funciona o algoritimo pela biblioteca [aqui](http://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic).

### SVD
O famoso algoritmo SVD , popularizado por Simon Funk durante o Prêmio Netflix. Quando as linhas de base não são usadas, isso é equivalente à fatoração de matrizes probabilísticas.
Mais detalhe sobre a formulação e como funciona o algoritimo pela biblioteca [aqui](http://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms).

In [6]:
sim_options = {
    'name': 'cosine',
    'user_based': True  # compute  similarities between items
}

algo = KNNBasic(sim_options=sim_options, k=4, min_k=2)
algo_svd = SVD()
for trainset, testset in pkf.split(data):

    # train and test algorithm.
    algo_svd.fit(trainset)
    algo.fit(trainset)
    predictions = algo.test(testset)
    predictions_svd = algo_svd.test(testset)
    accuracy.rmse(predictions,verbose=True)
    accuracy.rmse(predictions_svd,verbose=True)
    

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.1118
RMSE: 0.9472


## Métodos utilizados.
Abaixo estão os metodos que retornar o top 5 dos filmes de acordo com o SVD ou KNN e também o metodo que retornar o top 5 de usuarios de acordo com o KNN que possui perfil similar com o perfil selecionado na pesquisa. 

In [7]:
def get_top_5(uid):
    top = []
    items = not_watch[int(uid)]
    
    for item in items:
        top.append((item, algo.predict(uid=uid, iid=str(item)).est))
    
    return sorted(top, key=lambda item: item[1], reverse=True)[:5]


def get_top_5_movies_KNN(uid):
    top_5 = get_top_5(uid)
    return [item_data[int(item[0])][1] for item in top_5]

In [8]:
def get_top2_5(uid):
    top = []
    items = not_watch[int(uid)]
    
    for item in items:
        top.append((item, algo_svd.predict(uid=uid, iid=str(item)).est))
    
    return sorted(top, key=lambda item: item[1], reverse=True)[:5]

def get_top_5_movies_SVD(uid):
    top_5 = get_top2_5(uid)
    return [item_data[int(item[0])][1] for item in top_5]

In [9]:
def get_top_5_neighbors(uid):
    inner_uid = algo.trainset.to_inner_uid(uid)
    neighbords = algo.get_neighbors(iid=inner_uid, k=5)
    return [algo.trainset.to_raw_uid(iid) for iid in neighbords]

In [24]:
def get_top_5_cosine(uid):
    x = algo.sim
    y = get_top_5_neighbors(uid)
    user = int(uid)-1
    result = list()
    for i in range(5):
        result.append(x[user][int(y[i])-1])
    return result

In [34]:
def get_top_neighbors(uid):
    y = get_top_5_neighbors(uid)
    x = get_top_5_cosine(uid)
    result = list()
    for i in range(5):
        result.append("%.4f - %.4f" %(float(y[i]),float(x[i])))
    return result

## Exemplo da aplicação. Usuario id: 11. 

| Recomendações KNN | Recomendações SVD | Usuarios Proximos | Coseno |
| ------------- |:-------------:| :-----:| -----:| 
| Angels and Insects (1995) | Mighty Aphrodite (1995) | 9 |  1|
| Mother (1996)      | Maltese Falcon, The (1941) |   34 |  1|
| That Old Feeling (1997) | Ulee's Gold (1997)  |    86 |  1|
| Ayn Rand: A Sense of Life (1997) | Legends of the Fall (1994) |    88 |   1|
| Cure, The (1995) | Brazil (1985) |    93 | 1|

### Analise de Precisão dos algoritimos. 
| Algorimo | RMSE | 
| ------------- |:-------------:|
| KNN | 1.1118 |
| SVD| 0.9513 |

### Aplicação.
A aplicação encontrasse na rede com uma irteface onde pode se trabalhar com todos o usuário da base de dados assim o usuário só precisa escolher algum id e observar os resultados tanto do KNN quanto do SVD. Para mais detalhe acesse o site da aplicação [aqui.](https://sad-cloud.appspot.com/)